# Проект по ММТС -  Финансовый инструментарий в помощь продвинутым  игрокам на фондовом рынке

## План действий :
### 1) Получаем данные от московской биржи, пробуем разные временные периоды
### 2) Показываем все возможные, но выбираем 5 интересных акций для примера
### 3) Выгружаем по этим тикерам нужные данные
### 4) Реализуем Backtesting на примере нескольких классических стратегий
### 5) Оптимизация стратегий для конкретных Тикеров
### 6) Нотификации в браузере
### 7) Предсказания будущих цен (Prophet, ARIMA)  


### Получаем цены с Московской биржы с помощью их API и библиотеки pandas_datareader

In [4]:
!pip install apimoex

In [46]:
!pip install winotify

In [6]:
!pip install backtesting

  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173823 sha256=1f38a72a74730cbe973f7a5aafc00ecd052e810a08aca922283da2f427ac173b
  Stored in directory: c:\users\a.minakov\appdata\local\pip\cache\wheels\3f\7c\24\f8816cdb5359accfe50ebbb023baf41e98592f11528ed26ce6
Successfully built backtesting


In [9]:
!pip install pandas_datareader

In [1]:
import pandas as pd #Для работы с таблицами данных (дата фреймы)
import requests #Для запросов к серверу
import json #Для обработки ответов сервера

In [2]:
from ipywidgets import widgets  

In [7]:
import apimoex

from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

C:\Users\a.minakov\Anaconda3\lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [10]:
import datetime as dt
import pandas_datareader as web
import backtesting

In [11]:
# Получаем список всех возможных тикеров для акций и называем его allTickers

In [12]:
request_url = ('https://iss.moex.com/iss/engines/stock/'
               'markets/shares/boards/TQBR/securities.json')
arguments = {'securities.columns': ('SECID,'
                                    'SHORTNAME')}
with requests.Session() as session:
    iss = apimoex.ISSClient(session, request_url, arguments)
    data = iss.get()
    df_tick = pd.DataFrame(data['securities'])
    allTickers = list(df_tick.SECID.unique())
    print(df_tick)

     SECID   SHORTNAME
0     ABRD  АбрауДюрсо
1     ACKO     АСКО ао
2     AFKS  Система ао
3     AFLT    Аэрофлот
4     AGRO    AGRO-гдр
..     ...         ...
245   YNDX  Yandex clA
246   YRSB     ТНСэнЯр
247  YRSBP   ТНСэнЯр-п
248   ZILL      ЗИЛ ао
249   ZVEZ   ЗВЕЗДА ао

[250 rows x 2 columns]


In [13]:
for i in df_tick.values:
    print(i)

['ABRD' 'АбрауДюрсо']
['ACKO' 'АСКО ао']
['AFKS' 'Система ао']
['AFLT' 'Аэрофлот']
['AGRO' 'AGRO-гдр']
['AKRN' 'Акрон']
['ALRS' 'АЛРОСА ао']
['AMEZ' 'АшинскийМЗ']
['APTK' 'Аптеки36и6']
['AQUA' 'ИНАРКТИКА']
['ARSA' 'Арсагера']
['ASSB' 'АстрЭнСб']
['AVAN' 'Авангрд-ао']
['BANE' 'Башнефт ао']
['BANEP' 'Башнефт ап']
['BELU' 'Белуга ао']
['BISVP' 'БашИнСв ап']
['BLNG' 'Белон ао']
['BRZL' 'БурЗолото']
['BSPB' 'БСП ао']
['BSPBP' 'БСП ап']
['CBOM' 'МКБ ао']
['CHGZ' 'РН-ЗапСиб']
['CHKZ' 'ЧКПЗ ао']
['CHMF' 'СевСт-ао']
['CHMK' 'ЧМК ао']
['CIAN' 'CIAN-адр']
['CNTL' 'Телеграф']
['CNTLP' 'Телеграф-п']
['DIOD' 'ЗаводДИОД']
['DSKY' 'ДетскийМир']
['DVEC' 'ДЭК ао']
['DZRD' 'ДонскЗР']
['DZRDP' 'ДонскЗР п']
['EELT' 'ЕвроЭлтех']
['ELTZ' 'Электрцинк']
['ENPG' 'ЭН+ГРУП ао']
['ENRU' 'ЭнелРос ао']
['ETLN' 'ETLN-гдр']
['FEES' 'Россети']
['FESH' 'ДВМП ао']
['FIVE' 'FIVE-гдр']
['FIXP' 'FIXP-гдр']
['FLOT' 'Совкомфлот']
['GAZA' 'ГАЗ ао']
['GAZAP' 'ГАЗ ап']
['GAZC' 'ГАЗКОН-ао']
['GAZP' 'ГАЗПРОМ ао']
['GAZS' 'ГАЗ-серв

In [156]:
# Пусть нас интересуют 5 тикеров компаний из разных областей : Газпром, X5, Сбер, НорНикель, Яндекс

## Тут важная часть - определяем нужные данные

In [57]:
myTickers = widgets.SelectMultiple(
    options=df_tick["SECID"].values,
    description='Тикеры',
    disabled=False
)

display(myTickers)

SelectMultiple(description='Тикеры', options=('ABRD', 'ACKO', 'AFKS', 'AFLT', 'AGRO', 'AKRN', 'ALRS', 'AMEZ', …

In [58]:
myTickers.value

('FIVE', 'GAZP', 'GMKN', 'SBER', 'YNDX')

In [79]:
start_date = widgets.DatePicker(description = 'старт')
end_date = widgets.DatePicker(description = 'конец')
display(start_date)
display(end_date)

DatePicker(value=None, description='старт')

DatePicker(value=None, description='конец')

In [81]:
# Пользовательский ввод
upper_limits = [1800, 250, 20000, 120, 1500]
lower_limits = [1000, 180, 10000, 80, 1000]

In [82]:
## Пример простой стратегии, могу сделать более сложные примеры через talib
# Пользовательский ввод

class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20
    
    def init(self):
        # Precompute the two moving averages
        price = self.data.Close
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

In [83]:
strategy = widgets.Dropdown(
    options=[('SmaCross', SmaCross)],
    description='Strategy:',
    disabled=False,
)

display(strategy)

Dropdown(description='Strategy:', options=(('SmaCross', <class '__main__.SmaCross'>),), value=<class '__main__…

In [84]:
strategy = strategy.value

In [85]:
cash = widgets.IntText(
    description='Cash:',
    disabled=False
)

display(cash)

IntText(value=0, description='Cash:')

In [86]:
cash = cash.value

In [91]:
commission = widgets.FloatSlider(
    min=0,
    max=.5,
    step=0.01,
    description='Commission:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout_format='.2f',
)

display(commission)

FloatSlider(value=0.0, continuous_update=False, description='Commission:', max=0.5, step=0.01)

In [92]:
commission = commission.value

In [93]:
start = start_date.value
end = end_date.value

In [89]:
# Способ получения данных через apimoex
# with requests.Session() as session:
#     for tick in myTickers:
#         data = apimoex.get_board_history(session, tick, start = '2018-01-01')
#         pd.DataFrame(data).to_excel(f'{tick}_data.xlsx', index = False)
#         print(f'Данные {tick} получены и записаны')
#         print('------')

In [90]:
# Получение данных через DataReader, занимает дольше времени, но больше возможностей
for tick in myTickers.value:
    data = web.DataReader(tick, 'moex',start, end )
    pd.DataFrame(data).to_excel(f'{tick}_data.xlsx', index = True)
    print(f'Данные {tick} получены и записаны')
    print('------')

Данные FIVE получены и записаны
------
Данные GAZP получены и записаны
------
Данные GMKN получены и записаны
------
Данные SBER получены и записаны
------
Данные YNDX получены и записаны
------


In [94]:
def process_data(ticker):
    ticker = ticker.lower()
    data = pd.read_excel(f'{ticker}_data.xlsx', 
                         index_col = 0, 
                         usecols = ['TRADEDATE','VALUE','VOLUME','OPEN','HIGH','LOW','CLOSE'])
    data.columns = ['Close', 'High', 'Low', 'Open', 'Value', 'Volume']
    data.dropna(inplace = True)
    return data

In [95]:
data = {ticker: process_data(ticker) for ticker in myTickers.value}

In [96]:
for ticker, df in data.items(): 
    print(ticker)
    df.info()
    print("\n---------------------------\n")

FIVE
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1214 entries, 2018-02-01 to 2022-12-20
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   1214 non-null   float64
 1   High    1214 non-null   float64
 2   Low     1214 non-null   float64
 3   Open    1214 non-null   float64
 4   Value   1214 non-null   float64
 5   Volume  1214 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 66.4 KB

---------------------------

GAZP
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1237 entries, 2018-01-03 to 2022-12-20
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   1237 non-null   float64
 1   High    1237 non-null   float64
 2   Low     1237 non-null   float64
 3   Open    1237 non-null   float64
 4   Value   1237 non-null   float64
 5   Volume  1237 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 67.6 KB

-----------------

In [97]:
### Backtesting

In [99]:
myTickers.value

('FIVE', 'GAZP', 'GMKN', 'SBER', 'YNDX')

In [177]:
# Сначала запускаем тест стратегии с выбранными параметрами для всех тикеров,
# а потом оптимизируем для одной гиперпараметры, максимизируя конечную прибыль
# Можно показать разницу и сделать сравнительную таблицу (?)

In [40]:
# for data_source, tick in zip(data.values(), myTickers.value):
#     backtest = Backtest(data_source, SmaCross, cash=cash, commission = commission, exclusive_orders = True)
#     stats = backtest.run()
#     print(f'Stats for {tick}\n--------------')
#     print(stats)
#     print('\n---------------------------------------------\n')
#     backtest.plot(filename = f'{tick}_plot')

In [100]:
# Теперь для себа ещё раз просто выводим, а потом оптимизируем
for ticker, df in data.items():
    print(f'Stats for {ticker} before optimize\n--------------')
    bt = Backtest(df, strategy, cash=cash, commission=commission)
    stats = bt.run()
    bt.plot(filename = f'{ticker}_plot')
    print(stats)
    #print(stats['_trades'])
    print(f'\nStats for {ticker} after optimize\n--------------')
    stats = bt.optimize(n1=range(5, 30, 5),
                    n2=range(10, 70, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
    print(stats)
    print(stats._strategy)
    bt.plot(filename = f'{ticker}_best_plot')
    #print(stats['_equity_curve'])
    #print(stats['_trades'])
    print('\n---------------------------------------------\n')

Stats for FIVE before optimize
--------------
Start                     2018-02-01 00:00:00
End                       2022-12-20 00:00:00
Duration                   1783 days 00:00:00
Exposure Time [%]                   96.293245
Equity Final [$]                     446431.5
Equity Peak [$]                     1138010.0
Return [%]                          -55.35685
Buy & Hold Return [%]              -25.876777
Return (Ann.) [%]                  -15.414344
Volatility (Ann.) [%]               33.313643
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -64.686382
Avg. Drawdown [%]                  -12.129204
Max. Drawdown Duration     1560 days 00:00:00
Avg. Drawdown Duration      191 days 00:00:00
# Trades                                   68
Win Rate [%]                        45.588235
Best Trade [%]                      33.197556
Worst Trade [%]                   

  0%|          | 0/9 [00:00<?, ?it/s]

Start                     2018-02-01 00:00:00
End                       2022-12-20 00:00:00
Duration                   1783 days 00:00:00
Exposure Time [%]                   83.772652
Equity Final [$]                    1378560.0
Equity Peak [$]                     1597539.5
Return [%]                             37.856
Buy & Hold Return [%]              -25.876777
Return (Ann.) [%]                    6.891147
Volatility (Ann.) [%]               33.309727
Sharpe Ratio                         0.206881
Sortino Ratio                        0.337563
Calmar Ratio                         0.170187
Max. Drawdown [%]                  -40.491525
Avg. Drawdown [%]                   -8.612136
Max. Drawdown Duration     1180 days 00:00:00
Avg. Drawdown Duration       83 days 00:00:00
# Trades                                   21
Win Rate [%]                        38.095238
Best Trade [%]                      50.200089
Worst Trade [%]                    -16.144725
Avg. Trade [%]                    

  0%|          | 0/9 [00:00<?, ?it/s]

Start                     2018-01-03 00:00:00
End                       2022-12-20 00:00:00
Duration                   1812 days 00:00:00
Exposure Time [%]                   98.544867
Equity Final [$]                   3656267.62
Equity Peak [$]                    3656267.62
Return [%]                         265.626762
Buy & Hold Return [%]               20.226929
Return (Ann.) [%]                   30.227094
Volatility (Ann.) [%]               49.320962
Sharpe Ratio                         0.612865
Sortino Ratio                        1.487333
Calmar Ratio                          0.63451
Max. Drawdown [%]                  -47.638461
Avg. Drawdown [%]                   -6.178586
Max. Drawdown Duration     1210 days 00:00:00
Avg. Drawdown Duration       61 days 00:00:00
# Trades                                  126
Win Rate [%]                        44.444444
Best Trade [%]                      36.578354
Worst Trade [%]                    -18.404908
Avg. Trade [%]                    

...

Start                     2018-01-03 00:00:00
End                       2022-12-20 00:00:00
Duration                   1812 days 00:00:00
Exposure Time [%]                   96.847211
Equity Final [$]                    1794748.0
Equity Peak [$]                     1898692.0
Return [%]                            79.4748
Buy & Hold Return [%]               31.091842
Return (Ann.) [%]                   12.653646
Volatility (Ann.) [%]               33.089935
Sharpe Ratio                         0.382402
Sortino Ratio                        0.658467
Calmar Ratio                         0.437983
Max. Drawdown [%]                  -28.890728
Avg. Drawdown [%]                   -8.185576
Max. Drawdown Duration      338 days 00:00:00
Avg. Drawdown Duration       67 days 00:00:00
# Trades                                   39
Win Rate [%]                        41.025641
Best Trade [%]                       22.38806
Worst Trade [%]                     -8.343126
Avg. Trade [%]                    

  0%|          | 0/9 [00:00<?, ?it/s]

Start                     2018-01-03 00:00:00
End                       2022-12-20 00:00:00
Duration                   1812 days 00:00:00
Exposure Time [%]                   97.655618
Equity Final [$]                    3491634.6
Equity Peak [$]                    4244484.38
Return [%]                          249.16346
Buy & Hold Return [%]               -40.37085
Return (Ann.) [%]                   29.010514
Volatility (Ann.) [%]               45.726637
Sharpe Ratio                         0.634434
Sortino Ratio                        1.368035
Calmar Ratio                         0.862323
Max. Drawdown [%]                  -33.642294
Avg. Drawdown [%]                   -5.281792
Max. Drawdown Duration      572 days 00:00:00
Avg. Drawdown Duration       35 days 00:00:00
# Trades                                   83
Win Rate [%]                        45.783133
Best Trade [%]                        40.1766
Worst Trade [%]                    -10.756782
Avg. Trade [%]                    

  0%|          | 0/9 [00:00<?, ?it/s]

Start                     2018-01-03 00:00:00
End                       2022-12-20 00:00:00
Duration                   1812 days 00:00:00
Exposure Time [%]                   95.137763
Equity Final [$]                    1717206.7
Equity Peak [$]                     2339315.7
Return [%]                           71.72067
Buy & Hold Return [%]               -5.222537
Return (Ann.) [%]                   11.674507
Volatility (Ann.) [%]               39.323133
Sharpe Ratio                         0.296886
Sortino Ratio                         0.49495
Calmar Ratio                         0.266477
Max. Drawdown [%]                  -43.810576
Avg. Drawdown [%]                   -8.760552
Max. Drawdown Duration      650 days 00:00:00
Avg. Drawdown Duration       68 days 00:00:00
# Trades                                   39
Win Rate [%]                        35.897436
Best Trade [%]                      76.635584
Worst Trade [%]                    -19.886493
Avg. Trade [%]                    

### 6) Нотификации в браузере

In [47]:
import os
import time

In [48]:
from winotify import Notification, audio

In [184]:
# Делаем запрос каждые n (сейчас 10) секунд, сравниваем последнюю цену с границами

In [49]:
start=dt.datetime(2022,12,20)

In [53]:
myTickers.value

('FIVE', 'GAZP', 'GMKN', 'SBER', 'YNDX')

In [63]:
while True:
    
    last_prices = [web.DataReader(ticker, 'moex', start=start)['CLOSE'][-1] for ticker in myTickers.value]
    print(last_prices)
    for i in range(len(myTickers.value)):
        if last_prices[i] > upper_limits[i]:
            print(f'Цена {myTickers.value[i]} превышает верхнюю границу ')
            toast = Notification(app_id = 'MMTS Project',
                                title = f'Stock price alert for {myTickers.value[i]}',
                                msg = f' Цена {myTickers.value[i]} превысила критическое значение, достигнув {last_prices[i]}',
                                icon = os.path.join(os.getcwd(),f'{myTickers.value[i]}.png'),
                                duration = 'long')
            toast.add_actions(label = 'Перейти на страницу к брокеру',
                             launch = f'https://www.tinkoff.ru/invest/stocks/{myTickers.value[i]}/')
            toast.set_audio(audio.Default, loop = True)
            toast.show()
        elif last_prices[i] < lower_limits[i]:
            print(f'Цена {myTickers.value[i]} упала ниже минимальной границы')
            toast = Notification(app_id = 'MMTS Project',
                                title = f'Stock price alert for {myTickers.value[i]}',
                                msg = f' Цена {myTickers.value[i]} упала ниже выставленной границы, достигнув {last_prices[i]}',
                                icon = os.path.join(os.getcwd(),f'{myTickers.value[i]}.png'),
                                duration = 'long')
            toast.add_actions(label = 'Перейти на страницу к брокеру',
                             launch = f'https://www.tinkoff.ru/invest/stocks/{myTickers.value[i]}/')
            toast.set_audio(audio.Default, loop = True)
            toast.show()
        time.sleep(30) # Пользовательский ввод

[1592.0, 157.95, 14958.0, 137.69, 1847.0]
Цена GAZP упала ниже минимальной границы


KeyboardInterrupt: 